# Evaluation of linear probing/finetuning models on EMBED dataset

In [ ]:
import sys
import seaborn as sns

sys.path.append("/vol/biomedic3/mb121/causal-contrastive")

from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from classification.classification_module import ClassificationModule
from hydra import compose, initialize

from data_handling.mammo import EmbedDataModule, modelname_map
from evaluation.helper_functions import run_inference
rev_model_map = {v: k for k, v in modelname_map.items()}
import os

os.chdir("/vol/biomedic3/mb121/causal-contrastive/evaluation")

In [ ]:
# Mapping from human readable run name to Weights&Biases run_id. 

# Human readable name should be in format:
# for finetuning:
# {simclr/simclrcf/simclrcfaug}-{train_prop}-{seed}
# for linear probing
# {simclr/simclrcf/simclrcfaug}head-{train_prop}-{seed}

model_dict = {
    "supervised-0.01-11": "pxt8qa9a",
    "supervised-0.01-22": "pejcf0g2",
    "supervised-0.01-33": "hanrntpx",
    "supervised-0.05-11": "tovoj1oa",
    "supervised-0.05-22": "36xkkga3",
    "supervised-0.05-33": "h5fg9j3v",
    "supervised-0.1-22": "cnmbk18a",
    "supervised-0.1-11": "zyni8hft",
    "supervised-0.1-33": "bhvd9v9r",
    "supervised-0.25-11": "qdbyxgq6",
    "supervised-0.25-22": "kzazkpjz",
    "supervised-0.25-33": "gh6adhz5",
    "supervised-1.0-33": "u1zedefy",
    "supervised-1.0-11": "ynv4elgn",
    "supervised-1.0-22": "qdjpdsbu",
    "simclrhead-1.0-22": "c23ucrut",
    "simclrhead-1.0-33": "al1yw5g1",
    "simclrhead-0.01-22": "fcstcq4m",
    "simclrhead-0.05-22": "4rkb07qp",
    "simclrhead-0.1-22": "4kdbw3tn",
    "simclrhead-0.01-11": "qehq8ywl",
    "simclrhead-0.05-11": "szkv92wk",
    "simclrhead-0.1-11": "xi9yfyj1",
    "simclrhead-0.01-33": "9ot9ul0s",
    "simclrhead-0.05-33": "q2txgcbe",
    "simclrhead-0.1-33": "wtn08qbg",
    "simclrhead-0.25-22": "43db30f0",
    "simclrhead-0.25-11": "hdml1vz8",
    "simclrhead-0.25-33": "yxz6ixyc",
    "simclrcfhead-0.01-22": "q1wgmpaw",
    "simclrcfhead-0.05-22": "00v5peyb",
    "simclrcfhead-0.1-22": "lgmuodot",
    "simclrcfhead-0.01-11": "fwyq91oj",
    "simclrcfhead-0.05-11": "jsp93kpp",
    "simclrcfhead-0.1-11": "ggir6eh2",
    "simclrcfhead-0.01-33": "l7qee17l",
    "simclrcfhead-0.05-33": "d31qhxby",
    "simclrcfhead-0.1-33": "yuxiylvu",
    "simclrcfhead-0.25-22": "i497ktks",
    "simclrcfhead-0.25-11": "3utemt8e",
    "simclrcfhead-0.25-33": "vmazb0lu",
    "simclrcfhead-1.0-33": "apbnc0xw",
    "simclrcfhead-1.0-11": "094g4tym",
    "simclrcfhead-1.0-22": "o0hrjhbr",
    "simclrcf-0.01-11": "um1y3iln",
    "simclrcf-0.05-22": "stoqfdld",
    "simclr-0.05-22": "pf0uqv2e",
    "simclrcf-0.1-22": "jk0xk1yq",
    "simclr-0.05-11": "j7qg2zcf",
    "simclrcf-0.05-11": "ju9xfu6z",
    "simclr-0.1-22": "mh4exarz",
    "simclr-0.1-11": "3a1wft8u",
    "simclrcf-0.25-22": "w1muwi2v",
    "simclrcf-0.1-11": "2c8x9gr9",
    "simclr-0.25-22": "io9ukhfm",
    "simclrcf-0.05-33": "kkcbrb3i",
    "simclr-0.25-11": "tkr8tw5e",
    "simclrcf-0.25-11": "e8w1vxmn",
    "simclrcf-0.1-33": "bzyqmycb",
    "simclrcfaug-0.05-22": "z7pgwz4e",
    "simclrcfaug-0.05-11": "o98equwh",
    "simclrcfaug-0.05-33": "idly5nkx",
    "simclrcfaughead-0.05-11": "0nke7fpq",
    "simclrcfaug-0.1-22": "ed1lg5j2",
    "simclrcfaug-0.1-11": "huouaf2h",
    "simclrcfaug-0.1-33": "glifyzd8",
    "simclrcfaug-0.25-22": "boijfg9v",
    "simclrcfaughead-0.1-11": "9qt391fv",
    "simclrcf-1.0-22": "0cf2wpg6",
    "simclr-1.0-22": "qz7qx7u3",
    "simclrcfaug-0.25-11": "gw92xdg1",
    "simclrcf-1.0-11": "012vx3f7",
    "simclr-1.0-11": "md4ymp1g",
    "simclrcfaug-0.25-33": "9tmsbgp8",
    "simclrcfaughead-0.25-11": "8bfp44f7",
    "simclrcf-0.25-33": "3k1lu1oy",
    "simclrcfaug-1.0-22": "7lxph9ps",
    "simclrcfaughead-1.0-11": "9tg73cs0",
    "simclrcfaughead-0.05-22": "muyzcrq3",
    "simclrcfaug-1.0-33": "u4onu6om",
    "simclrcfaughead-0.1-22": "jn0pdyu2",
    "simclr-0.01-33": "hsyrkrpi",
    "simclr-0.01-22": "nsr3ixey",
    "simclr-0.01-11": "cpqdq4gs",
    "simclrcfaughead-0.25-22": "o95101nl",
    "simclrcf-0.01-22": "qxvdehkn",
    "simclrcf-0.01-33": "nrz507vz",
    "simclrcfaug-0.01-11": "7heqiv3l",
    "simclrcfaug-0.01-22": "iw4v9i4d",
    "simclrcfaug-0.01-33": "hcabqnze",
    "simclrcf-1.0-33": "lhaovv2s",
    "simclr-0.05-33": "eissogv0",
    "simclr-0.1-33": "1wwpkejb",
    "simclr-0.25-33": "gx6u8o0b",
    "simclrcfaughead-0.01-11": "r1l5h6wx",
    "simclrcfaughead-0.01-22": "qo5hrfi3",
    "simclrcfaughead-0.01-33": "9ghb5o2a",
    "simclrcfaughead-1.0-22": "eanz6j03",
    "simclrcfaughead-0.05-33": "05nbp9zw",
    "simclrcfaughead-0.1-33": "mlboruuc",
    "simclrcfaughead-0.25-33": "ki5t0xzh",
    "simclrcfaughead-1.0-33": "8a9y439k",
    "simclr-1.0-33": "d1zvpt88",
    "simclrhead-1.0-11": "g4no8azy",
    "simclrcfaug-1.0-11": "b09dget0",
}

In [ ]:
with initialize(version_base=None, config_path="../configs"):
    cfg = compose(
        config_name="config.yaml",
        overrides=["experiment=base_density", "data.cache=False", "seed=22"],
    )
    print(cfg)
    data_module = EmbedDataModule(config=cfg)
test_loader = data_module.test_dataloader()

In [ ]:
df = pd.read_csv(f"../outputs/classification_{cfg.data.label}_results_finetune2.csv")
for run_name, run_id in model_dict.items():
    already_in_df = run_name in df.run_name.values
    if run_id != "" and not already_in_df:
        print(run_name)
        model_to_evaluate = f"../outputs/run_{run_id}/best.ckpt"
        classification_model = ClassificationModule.load_from_checkpoint(
            model_to_evaluate, map_location="cuda"
        ).model.eval()
        classification_model.cuda()

        # ID evaluation
        inference_results = run_inference(test_loader, classification_model)
        scanners = np.argmax(inference_results["scanners"], 1)

        for i in np.unique(scanners):
            print(f"\nEvaluating scanner {i}")
            sc_idx = np.where(scanners == i)
            targets = inference_results["targets"][sc_idx]
            preds = np.argmax(inference_results["confs"], 1)[sc_idx]
            confs = inference_results["confs"][sc_idx]
            res = {}
            res["N_test"] = [targets.shape[0]]
            res["ROC"] = [roc_auc_score(targets, confs, multi_class="ovr")]
            res["Model Name"] = [rev_model_map[i]]
            res["run_name"] = run_name
            print(res)
            df = pd.concat([df, pd.DataFrame(res, index=[0])], ignore_index=True)

        df.to_csv(
            f"../outputs/classification_{cfg.data.label}_results_finetune2.csv",
            index=False,
        )

## Plotting embeddings

In [ ]:
model_dict_for_embeddings = {
    "SimCLR": "byatk1eo",
    "SimCLR with CF in training set": "jv9hzx89",
    "Counterfactual SimCLR": "kywspwfs",
}

In [ ]:
from evaluation.helper_functions import run_get_embeddings
from sklearn.manifold import TSNE

results = {}

for run_name, run_id in model_dict_for_embeddings.items():
    if run_id != "":
        print(run_name)
        model_to_evaluate = f"../outputs/run_{run_id}/epoch=449.ckpt"
        classification_model = ClassificationModule.load_from_checkpoint(
            model_to_evaluate, map_location="cuda"
        ).model.eval()
        classification_model.cuda()
        # ID evaluation
        inference_results = run_get_embeddings(
            test_loader, classification_model, max_batch=500
        )
        inference_results["scanners"] = np.argmax(inference_results["scanners"], 1)
        results[run_name] = inference_results

In [ ]:

f, ax = plt.subplots(1, len(results.keys()), figsize=(20, 4))
for i, (run_name, inference_results) in enumerate(results.items()):
    scanners = inference_results["scanners"]
    idx = np.where(scanners != -1)[0]
    targets = inference_results["targets"]
    tsne = TSNE(n_jobs=6, perplexity=30)
    x2d = tsne.fit_transform(inference_results["feats"][idx])
    scanners_plot = [rev_model_map[s] for s in scanners[idx]]
    sns.scatterplot(
        x=x2d[idx, 0],
        y=x2d[idx, 1],
        hue=scanners_plot,
        ax=ax[i],
        palette=sns.color_palette("rocket"),
        legend=i == 1,
        alpha=1.0,
        s=30,
    )
    ax[i].set_xlabel("")
    ax[i].set_ylabel("")
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    ax[i].set_title(run_name, fontsize=18)
ax[1].legend(
    loc="center", bbox_to_anchor=(0.5, -0.15), ncol=6, fontsize=16, markerscale=2
)
plt.savefig("embeddings_id_tsne.pdf", bbox_inches="tight")
plt.savefig("embeddings_id_tsne.png", bbox_inches="tight", dpi=500)